In [1]:
import math as mt
import numpy as np
import pandas as pd
from scipy.stats import norm

# Baseline
## input baseline

In [2]:
from IPython.display import IFrame
IFrame('https://pic3.zhimg.com/80/v2-be6105a8adf7b30cbdd8a072c8dfd38a_720w.jpg', width=800, height=340)

In [3]:
baseline = {
    'Cookies' : 40000,
    'Clicks' : 3200,
    'Enrollemnts' : 660,
    'CTP' : 0.08,
    'GConversion' : 0.2063,
    'NConversion' : 0.109313,
    'Retention' : 0.53
}

In [4]:
IFrame('https://pic3.zhimg.com/80/v2-3ff8fafd3ca04ca32352788bb5caad36_720w.jpg', width=800, height=340)

## change the baseline according the sample size

In [5]:
baseline['Cookies'] = 5000
baseline['Clicks'] = baseline['Clicks'] * (5000/40000)
baseline['Enrollemnts'] = baseline['Enrollemnts'] * (5000/40000)
baseline

{'Cookies': 5000,
 'Clicks': 400.0,
 'Enrollemnts': 82.5,
 'CTP': 0.08,
 'GConversion': 0.2063,
 'NConversion': 0.109313,
 'Retention': 0.53}

# Calculate sd

In [6]:
conversion_dict = {
    'GConversion' : [0.01, 'Clicks'],   #第一个对应d_min,第二个对应分母
    'NConversion' : [0.0075, 'Clicks']
}

def cal_sd(conversion, d_min, denominator):
    R = {}
    R['d_min'] = d_min
    R['p'] = baseline[conversion]
    R['n'] = baseline[denominator]
    R['sd'] = round(mt.sqrt((R['p']*(1-R['p'])) / R['n']), 4)
    print(conversion, 'Standard Deviation: ', R['sd'])
    
conversions = ['GConversion', 'NConversion']
for conversion in conversions:
    d_min = conversion_dict[conversion][0]
    denominator = conversion_dict[conversion][1]
    cal_sd(conversion, d_min, denominator)

GConversion Standard Deviation:  0.0202
NConversion Standard Deviation:  0.0156


# Calculate the sample size

## using online calculator

In [7]:
from IPython.display import IFrame
IFrame('https://www.evanmiller.org/ab-testing/sample-size.html#!20;80;5;5;0', width=800, height=500)

## using Python to calculate

In [8]:
#计算sd1和sd2
def get_sds(p,d):
    sd1=mt.sqrt(2*p*(1-p))
    sd2=mt.sqrt(p*(1-p)+(p+d)*(1-(p+d)))
    x=[sd1,sd2]
    return x

#计算z-score
def get_z_score(alpha):
    return norm.ppf(alpha)

#计算样本量
def get_sampSize(sds,alpha,beta,d):
    n=pow((get_z_score(1-alpha/2)*sds[0]+get_z_score(1-beta)*sds[1]),2)/pow(d,2)
    return n

#指定基线数据p和d
GC={}
GC["p"]=baseline["GConversion"]
GC["d"]=0.01
NC={}
NC["p"]=baseline["NConversion"]
NC["d"]=0.0075

GC["SampSize"]=round(get_sampSize(get_sds(GC["p"],GC["d"]),0.05,0.2,GC["d"]))
print(GC["SampSize"])
GC["SampSize"]=round(GC["SampSize"]/0.08*2)
print(GC["SampSize"])

NC["SampSize"]=round(get_sampSize(get_sds(NC["p"],NC["d"]),0.05,0.2,NC["d"]))
print(NC["SampSize"])
NC["SampSize"]=NC["SampSize"]/0.08*2
print(NC["SampSize"])

25839
645975
27413
685325.0


# Import data

In [9]:
control = pd.read_csv('control_data.csv')
experiment = pd.read_csv('experiment_data.csv')
control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [10]:
print("Control:")
print(control.sum())
print("\nExperiment:")
print(experiment.sum())

Control:
Date           Sat, Oct 11Sun, Oct 12Mon, Oct 13Tue, Oct 14We...
Pageviews                                                 345543
Clicks                                                     28378
Enrollments                                               3785.0
Payments                                                  2033.0
dtype: object

Experiment:
Date           Sat, Oct 11Sun, Oct 12Mon, Oct 13Tue, Oct 14We...
Pageviews                                                 344660
Clicks                                                     28325
Enrollments                                               3423.0
Payments                                                  1945.0
dtype: object


# Check robustness
## Check pageview
### Pageview

###### Method 1

In [12]:
pageview_control = control['Pageviews'].sum()
pageview_experiment = experiment['Pageviews'].sum()
pageview_total = pageview_control + pageview_experiment

p = 0.5
alpha = 0.05
p_hat = round(pageview_control / pageview_total, 4)
sd = mt.sqrt(p*(1-p) / pageview_total)
ME = round(get_z_score(1 - (alpha/2)) * sd, 4)
CI = np.array([p-ME, p+ME])
print('Confidence Interval : ', CI)

Confidence Interval :  [0.4988 0.5012]


###### Method 2

In [13]:
mean = pageview_total * p
sd = mt.sqrt(pageview_total * p * (1-p))
z = get_z_score(1 - (alpha/2))
left = round(mean - z*sd, 0)
right = round(mean + z*sd, 0)
CI = np.array([left, right])
print('Confidence Interval : ', CI)
print(pageview_control, pageview_experiment)

Confidence Interval :  [344287. 345916.]
345543 344660


### Click

In [15]:
click_control = control['Clicks'].sum()
click_experiment = experiment['Clicks'].sum()
click_total = click_control + click_experiment

p = 0.5
alpha = 0.05
p_hat = round(click_control / click_total, 4)
sd = mt.sqrt(p*(1-p) / click_total)
ME = round(get_z_score(1 - (alpha/2)) * sd, 4)
CI = np.array([p-ME, p+ME])
print('Confidence Interval : ', CI)

Confidence Interval :  [0.4959 0.5041]


### CTP( clicks / pageviews)

In [48]:
ctp_control = click_control / pageview_control
ctp_experiment = click_experiment / pageview_experiment
d_hat = round(ctp_control-ctp_experiment,4)
p_pooled=click_total/pageview_total
sd_pooled=mt.sqrt(p_pooled*(1-p_pooled)*(1/pageview_control+1/pageview_experiment))
ME=round(get_z_score(1-(alpha/2))*sd_pooled,4)
print ("置信区间的范围在",0-ME,"和",0+ME,"之间，样本值是",d_hat)

置信区间的范围在 -0.0013 和 0.0013 之间，样本值是 -0.0001


# Check core index

In [ ]:
clicks_cont=control["Clicks"].loc[control["Enrollments"].notnull()].sum()
clicks_exp=experiment["Clicks"].loc[experiment["Enrollments"].notnull()].sum()

enrol_control = control['Enrollments'].sum()
enrol_experiment = experiment['Enrollments'].sum()
GConversion_control = enrol_control / clicks_cont
GConversion_experiment = enrol_experiment / clicks_exp
d_hat = round(GConversion_control - GConversion_experiment, 4)
p_pooled = (enrol_control + enrol_experiment) / (clicks_cont + clicks_exp)
sd_pooled = mt.sqrt(p_pooled * (1-p_pooled) * (1/clicks_cont + 1/clicks_exp))
ME=round(get_z_score(1-(alpha/2))*sd_pooled,4)
print ("置信区间的范围在",d_hat-ME,"和",d_hat+ME,"之间，样本值是",d_hat)

print(GConversion_control, GConversion_experiment)

In [ ]:

print(clicks_cont, clicks_exp)
print(click_control, click_experiment)